In [7]:
import json
import numpy as np
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
import functions as f
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import pyLDAvis.gensim as gensimvis
import pyLDAvis
from gensim import corpora

In [2]:
with open('/home/nick/Documents/Reddit_NLP/askreddit_lemmatized_dict.pickle', 'rb') as file:
    text = pickle.load(file)

In [3]:
stop_list = ['of', 'it', 'to', 'for', 'you', 'the', 'that', 'if', 'my', 'and', 'in', 'from','going', 'too', 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaaaaaaaaaaaaaa', 'aaaaaaaaaaaa', 'aaaaaaaa', 'aaa', 'aaaaaaaaahhhhhhhhhhhh', 'aaaaaaaand', 'aaaaaaaauuuuuugggghhh', 'aaaaaaafternoon', 'aaaaaaand', 'aaaaaand', 'aaaaall', 'aaaaalllllll', 'aaaaand', 'is', 'are', 'what', 'they', 'we', 'about', 'me', 'have', 'im', 'this', 'yeah','will','can','get','be', 'have','be', 'but', 'he', 'she', 'her', 'was', 'with', 'they']

cvn = TfidfVectorizer(stop_words=stop_list)
data_cvn = cvn.fit_transform(text)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
# data_dtmn.columns = data_df.index
data_dtmn

,0,1,2,3,4,5,6,7,8,9,...,101131,101132,101133,101134,101135,101136,101137,101138,101139,101140
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaah,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaahwooooo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaalll,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝙁𝙄𝙓,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝙃𝙀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝙄𝙏,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝙏𝙃𝙀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
word2id = dict((k, v) for k, v in cvn.vocabulary_.items())

In [5]:
ldan = models.LdaMulticore(corpus=corpusn, num_topics=20, id2word=id2wordn, passes=5)
ldan.print_topics()

INFO - 20:37:49: using symmetric alpha at 0.05
INFO - 20:37:49: using symmetric eta at 0.05
INFO - 20:37:49: using serial LDA version on this node
INFO - 20:37:49: running online LDA training, 20 topics, 5 passes over the supplied corpus of 101141 documents, updating every 30000 documents, evaluating every ~101141 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 20:37:49: training LDA model using 15 processes
INFO - 20:37:50: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/101141, outstanding queue size 1
INFO - 20:37:50: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/101141, outstanding queue size 2
INFO - 20:37:50: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/101141, outstanding queue size 3
INFO - 20:37:50: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/101141, outstanding queue size 4
INFO - 20:37:50: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/101141, outstanding queue size 5
IN

INFO - 20:37:55: merging changes from 30000 documents into a model of 101141 documents
INFO - 20:37:55: topic #6 (0.050): 0.012*"not" + 0.007*"people" + 0.005*"like" + 0.005*"talk" + 0.005*"time" + 0.005*"think" + 0.005*"work" + 0.004*"thing" + 0.004*"know" + 0.004*"life"
INFO - 20:37:55: topic #9 (0.050): 0.009*"not" + 0.007*"trump" + 0.005*"like" + 0.004*"time" + 0.004*"think" + 0.003*"people" + 0.003*"woman" + 0.003*"good" + 0.003*"know" + 0.003*"want"
INFO - 20:37:55: topic #5 (0.050): 0.009*"not" + 0.005*"like" + 0.005*"good" + 0.005*"time" + 0.004*"think" + 0.004*"go" + 0.003*"thing" + 0.003*"girl" + 0.003*"know" + 0.003*"people"
INFO - 20:37:55: topic #15 (0.050): 0.008*"not" + 0.005*"time" + 0.005*"think" + 0.004*"know" + 0.004*"like" + 0.004*"sex" + 0.004*"bob" + 0.004*"year" + 0.004*"man" + 0.004*"wife"
INFO - 20:37:55: topic #0 (0.050): 0.008*"not" + 0.006*"like" + 0.005*"people" + 0.005*"high" + 0.005*"hitler" + 0.005*"think" + 0.005*"good" + 0.004*"time" + 0.004*"year" + 0

INFO - 20:38:02: topic #17 (0.050): 0.011*"not" + 0.007*"think" + 0.006*"tell" + 0.006*"yes" + 0.006*"life" + 0.006*"valentine" + 0.005*"funny" + 0.005*"good" + 0.005*"time" + 0.005*"like"
INFO - 20:38:02: topic diff=0.038785, rho=0.137921
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #35 = documents up to #72000/101141, outstanding queue size 5
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #36 = documents up to #74000/101141, outstanding queue size 4
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #37 = documents up to #76000/101141, outstanding queue size 5
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #38 = documents up to #78000/101141, outstanding queue size 4
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #39 = documents up to #80000/101141, outstanding queue size 5
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #40 = documents up to #82000/101141, outstanding queue size 6
INFO - 20:38:02: PROGRESS: pass 1, dispatched chunk #41 = documents up to #8

INFO - 20:38:09: PROGRESS: pass 2, dispatched chunk #25 = documents up to #52000/101141, outstanding queue size 4
INFO - 20:38:10: PROGRESS: pass 2, dispatched chunk #26 = documents up to #54000/101141, outstanding queue size 5
INFO - 20:38:11: PROGRESS: pass 2, dispatched chunk #27 = documents up to #56000/101141, outstanding queue size 4
INFO - 20:38:11: PROGRESS: pass 2, dispatched chunk #28 = documents up to #58000/101141, outstanding queue size 3
INFO - 20:38:12: PROGRESS: pass 2, dispatched chunk #29 = documents up to #60000/101141, outstanding queue size 3
INFO - 20:38:12: PROGRESS: pass 2, dispatched chunk #30 = documents up to #62000/101141, outstanding queue size 4
INFO - 20:38:12: PROGRESS: pass 2, dispatched chunk #31 = documents up to #64000/101141, outstanding queue size 5
INFO - 20:38:12: PROGRESS: pass 2, dispatched chunk #32 = documents up to #66000/101141, outstanding queue size 6
INFO - 20:38:12: PROGRESS: pass 2, dispatched chunk #33 = documents up to #68000/101141,

INFO - 20:38:16: PROGRESS: pass 3, dispatched chunk #17 = documents up to #36000/101141, outstanding queue size 5
INFO - 20:38:16: PROGRESS: pass 3, dispatched chunk #18 = documents up to #38000/101141, outstanding queue size 6
INFO - 20:38:16: PROGRESS: pass 3, dispatched chunk #19 = documents up to #40000/101141, outstanding queue size 7
INFO - 20:38:16: merging changes from 30000 documents into a model of 101141 documents
INFO - 20:38:16: topic #9 (0.050): 0.009*"trump" + 0.008*"not" + 0.006*"agree" + 0.005*"like" + 0.005*"john" + 0.004*"masturbate" + 0.004*"fictional" + 0.004*"kid" + 0.004*"people" + 0.004*"think"
INFO - 20:38:16: topic #5 (0.050): 0.010*"book" + 0.008*"read" + 0.008*"not" + 0.007*"thanks" + 0.006*"congrat" + 0.005*"like" + 0.005*"emperor" + 0.004*"good" + 0.004*"prime" + 0.004*"think"
INFO - 20:38:16: topic #7 (0.050): 0.011*"fuck" + 0.011*"live" + 0.008*"not" + 0.007*"good" + 0.007*"play" + 0.006*"oh" + 0.005*"like" + 0.004*"know" + 0.004*"hell" + 0.004*"give"
IN

INFO - 20:38:20: -13.707 per-word bound, 13373.7 perplexity estimate based on a held-out corpus of 1141 documents with 2465 words
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #0 = documents up to #2000/101141, outstanding queue size 1
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #1 = documents up to #4000/101141, outstanding queue size 2
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #2 = documents up to #6000/101141, outstanding queue size 3
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #3 = documents up to #8000/101141, outstanding queue size 4
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #4 = documents up to #10000/101141, outstanding queue size 5
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #5 = documents up to #12000/101141, outstanding queue size 6
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #6 = documents up to #14000/101141, outstanding queue size 6
INFO - 20:38:20: PROGRESS: pass 4, dispatched chunk #7 = documents up to #16000/101

INFO - 20:38:55: merging changes from 37141 documents into a model of 101141 documents
INFO - 20:38:55: topic #7 (0.050): 0.012*"fuck" + 0.011*"live" + 0.008*"good" + 0.008*"not" + 0.007*"play" + 0.007*"oh" + 0.005*"andrew" + 0.005*"hell" + 0.005*"disgusting" + 0.004*"game"
INFO - 20:38:55: topic #3 (0.050): 0.036*"thank" + 0.009*"alive" + 0.008*"congratulation" + 0.007*"cat" + 0.006*"carlin" + 0.006*"not" + 0.006*"pant" + 0.005*"like" + 0.005*"wow" + 0.005*"man"
INFO - 20:38:55: topic #5 (0.050): 0.011*"book" + 0.009*"read" + 0.008*"thanks" + 0.008*"emperor" + 0.007*"congrat" + 0.007*"not" + 0.006*"prime" + 0.005*"like" + 0.004*"reference" + 0.004*"good"
INFO - 20:38:55: topic #0 (0.050): 0.016*"love" + 0.011*"heel" + 0.010*"hitler" + 0.010*"not" + 0.010*"high" + 0.007*"like" + 0.007*"god" + 0.007*"dog" + 0.006*"good" + 0.005*"think"
INFO - 20:38:55: topic #6 (0.050): 0.019*"not" + 0.009*"people" + 0.009*"like" + 0.008*"life" + 0.007*"know" + 0.007*"want" + 0.007*"think" + 0.007*"thin

[(0,
  '0.016*"love" + 0.011*"heel" + 0.010*"hitler" + 0.010*"not" + 0.010*"high" + 0.007*"like" + 0.007*"god" + 0.007*"dog" + 0.006*"good" + 0.005*"think"'),
 (1,
  '0.015*"mr" + 0.009*"shrek" + 0.009*"ok" + 0.007*"not" + 0.006*"happen" + 0.005*"yoda" + 0.005*"haha" + 0.005*"poop" + 0.004*"think" + 0.004*"carl"'),
 (2,
  '0.010*"work" + 0.009*"nice" + 0.009*"not" + 0.007*"friend" + 0.006*"pay" + 0.005*"gay" + 0.005*"sure" + 0.005*"job" + 0.005*"lincoln" + 0.005*"year"'),
 (3,
  '0.036*"thank" + 0.009*"alive" + 0.008*"congratulation" + 0.007*"cat" + 0.006*"carlin" + 0.006*"not" + 0.006*"pant" + 0.005*"like" + 0.005*"wow" + 0.005*"man"'),
 (4,
  '0.020*"sex" + 0.011*"tell" + 0.011*"not" + 0.010*"jesus" + 0.010*"time" + 0.009*"lol" + 0.009*"girl" + 0.008*"like" + 0.007*"secret" + 0.007*"mom"'),
 (5,
  '0.011*"book" + 0.009*"read" + 0.008*"thanks" + 0.008*"emperor" + 0.007*"congrat" + 0.007*"not" + 0.006*"prime" + 0.005*"like" + 0.004*"reference" + 0.004*"good"'),
 (6,
  '0.019*"not" + 0.

In [8]:
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

INFO - 20:39:16: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO - 20:39:16: NumExpr defaulting to 8 threads.


In [9]:
with open(f'/home/nick/Documents/Reddit_NLP/pickles/wsb.pickle', 'rb') as file:
    wsb = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/science.pickle', 'rb') as file:
    science = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/ama.pickle', 'rb') as file:
    ama = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/writingprompts.pickle', 'rb') as file:
    askreddit = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/worldnews.pickle', 'rb') as file:
    worldnews = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/funny.pickle', 'rb') as file:
    funny = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/dankmemes.pickle', 'rb') as file:
    dankmemes = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/politics.pickle', 'rb') as file:
    politics = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/memes.pickle', 'rb') as file:
    memes = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/psychology.pickle', 'rb') as file:
    psychology = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/nosleep.pickle', 'rb') as file:
    nosleep = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/pickles/cscareerquestions.pickle', 'rb') as file:
    cscareerquestions = pickle.load(file)

In [10]:
all_others = cscareerquestions + nosleep + psychology + memes + politics + dankmemes + funny + worldnews + askreddit + ama + science + wsb

In [11]:
all_others

["Hi there,\n\nI 've been looking for a stable job (freelancing rn) for the past year in a FAANG company and right now am feeling pretty down because after all the work done (basically from august 2019 til now) I still cannot get an offer.  \nFor a bit of a background, I am a devops/sre engineer with 5 years of experience mostly in Asia in SME's. I have a master degree from a top tier engineering school in telecoms.  \nI started to interview a year ago with Nextdoor for SF and finished a few weeks ago with Facebook for London where I screwed up in a extra coding interview.  \nI've done not less than 14 full processes so far (mostly for London because of the visa issues for the states as I am not a US citizen):\n\n* Nextdoor (SF)\n* Google (Sydney)\n* Yelp (London)\n* Revolut (London)\n* Dataiku (London)\n* ThoughtWorks (London)\n* Cloudflare (London ) \\* 2\n* Glovo (Barcelona - offer)\n* Bulb (London)\n* Cisco + Cisco Meraki (London)\n* Apple (London)\n* Facebook (London)\n* Mastercar

In [12]:
clean_text = f.pre_clean(all_others)

In [13]:
lemmatized = f.lemmatize(clean_text)

In [14]:
lemmatized[0]

'hi   ve look stable job freelancing rn past year faang company right feel pretty work basically august   til offer    bit background devopssre engineer   year experience asia sme master degree tier engineering school telecom    start interview year ago nextdoor sf finish week ago facebook london screw extra coding interview    have   process far london visa issue state citizen    nextdoor sf   google sydney   yelp london   revolut london   dataiku london   thoughtworks london   cloudflare london      glovo barcelona   offer   bulb london   cisco   cisco meraki london   apple london   facebook london   mastercard london   question hard company have book read crack code interview code leetcode feel devop people require developer problem facebookapple read tanenbaum shit lowlevel os functionality refresh memory tcp packet   think covid currently impact way tech company recruit maybe picky lowerti tech company retry year life finally   feel especially hard devopssre peep company definitio

In [15]:
cvn = TfidfVectorizer(stop_words=stop_list)
data_cvn = cvn.fit_transform(lemmatized)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
data_dtmn

,0,1,2,3,4,5,6,7,8,9,...,195608,195609,195610,195611,195612,195613,195614,195615,195616,195617
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaaaaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaaaaaaaaaaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaaaaaaaaaaaaaaaaaaaaaaaaa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝘂𝘀𝗲𝗱,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝘄𝗮𝘀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝘆𝗲𝗮𝗿𝘀,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
𝚃𝚑𝚒𝚜,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
word2id = dict((k, v) for k, v in cvn.vocabulary_.items())

In [17]:
ldan = models.LdaMulticore(corpus=corpusn, num_topics=20, id2word=id2wordn, passes=5)
ldan.print_topics()

INFO - 21:56:53: using symmetric alpha at 0.05
INFO - 21:56:53: using symmetric eta at 0.05
INFO - 21:56:53: using serial LDA version on this node
INFO - 21:56:53: running online LDA training, 20 topics, 5 passes over the supplied corpus of 195618 documents, updating every 30000 documents, evaluating every ~195618 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 21:56:53: training LDA model using 15 processes
INFO - 21:56:57: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/195618, outstanding queue size 1
INFO - 21:56:57: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/195618, outstanding queue size 2
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/195618, outstanding queue size 3
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/195618, outstanding queue size 4
INFO - 21:56:58: PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/195618, outstanding queue size 3
IN

INFO - 21:57:08: topic #4 (0.050): 0.009*"not" + 0.007*"like" + 0.006*"think" + 0.006*"work" + 0.005*"good" + 0.004*"trump" + 0.004*"know" + 0.004*"look" + 0.004*"mask" + 0.003*"people"
INFO - 21:57:08: topic #14 (0.050): 0.010*"not" + 0.006*"like" + 0.005*"work" + 0.005*"vote" + 0.005*"know" + 0.005*"people" + 0.004*"think" + 0.004*"good" + 0.004*"look" + 0.004*"job"
INFO - 21:57:08: topic #1 (0.050): 0.009*"not" + 0.006*"think" + 0.006*"like" + 0.005*"work" + 0.004*"people" + 0.004*"thing" + 0.004*"go" + 0.004*"job" + 0.003*"good" + 0.003*"time"
INFO - 21:57:08: topic #0 (0.050): 0.009*"not" + 0.006*"like" + 0.004*"look" + 0.004*"think" + 0.004*"people" + 0.003*"work" + 0.003*"know" + 0.003*"company" + 0.003*"go" + 0.003*"click"
INFO - 21:57:08: topic diff=0.035436, rho=0.174078
INFO - 21:57:08: PROGRESS: pass 0, dispatched chunk #48 = documents up to #98000/195618, outstanding queue size 2
INFO - 21:57:08: PROGRESS: pass 0, dispatched chunk #49 = documents up to #100000/195618, outs

INFO - 21:57:13: PROGRESS: pass 0, dispatched chunk #94 = documents up to #190000/195618, outstanding queue size 7
INFO - 21:57:13: PROGRESS: pass 0, dispatched chunk #95 = documents up to #192000/195618, outstanding queue size 7
INFO - 21:57:13: PROGRESS: pass 0, dispatched chunk #96 = documents up to #194000/195618, outstanding queue size 7
INFO - 21:57:13: merging changes from 30000 documents into a model of 195618 documents
INFO - 21:57:14: topic #1 (0.050): 0.008*"not" + 0.006*"think" + 0.005*"like" + 0.004*"people" + 0.004*"go" + 0.004*"work" + 0.003*"thing" + 0.003*"time" + 0.003*"good" + 0.003*"know"
INFO - 21:57:14: topic #9 (0.050): 0.009*"not" + 0.007*"like" + 0.005*"work" + 0.005*"think" + 0.005*"people" + 0.004*"go" + 0.004*"time" + 0.004*"buy" + 0.003*"look" + 0.003*"good"
INFO - 21:57:14: topic #11 (0.050): 0.011*"not" + 0.006*"like" + 0.006*"know" + 0.004*"work" + 0.004*"people" + 0.004*"think" + 0.004*"look" + 0.004*"want" + 0.004*"mean" + 0.004*"mask"
INFO - 21:57:14:

INFO - 21:57:19: topic #3 (0.050): 0.011*"tesla" + 0.007*"nice" + 0.007*"not" + 0.007*"like" + 0.005*"click" + 0.004*"think" + 0.004*"look" + 0.004*"elon" + 0.003*"go" + 0.003*"iv"
INFO - 21:57:19: topic #0 (0.050): 0.009*"not" + 0.007*"hold" + 0.005*"like" + 0.005*"silver" + 0.004*"think" + 0.004*"exactly" + 0.004*"people" + 0.003*"sorry" + 0.003*"look" + 0.003*"know"
INFO - 21:57:19: topic #18 (0.050): 0.012*"not" + 0.005*"like" + 0.004*"work" + 0.004*"think" + 0.004*"wtf" + 0.004*"people" + 0.004*"trump" + 0.004*"agree" + 0.004*"right" + 0.004*"go"
INFO - 21:57:19: topic diff=0.033692, rho=0.100096
INFO - 21:57:19: PROGRESS: pass 1, dispatched chunk #36 = documents up to #74000/195618, outstanding queue size 7
INFO - 21:57:19: PROGRESS: pass 1, dispatched chunk #37 = documents up to #76000/195618, outstanding queue size 6
INFO - 21:57:19: PROGRESS: pass 1, dispatched chunk #38 = documents up to #78000/195618, outstanding queue size 7
INFO - 21:57:19: PROGRESS: pass 1, dispatched chu

INFO - 21:57:26: topic #14 (0.050): 0.010*"not" + 0.006*"like" + 0.005*"people" + 0.005*"vote" + 0.004*"think" + 0.004*"know" + 0.004*"work" + 0.003*"good" + 0.003*"look" + 0.003*"way"
INFO - 21:57:26: topic #4 (0.050): 0.009*"not" + 0.007*"mask" + 0.006*"like" + 0.006*"good" + 0.005*"think" + 0.004*"people" + 0.004*"wear" + 0.004*"know" + 0.004*"trump" + 0.004*"work"
INFO - 21:57:26: topic #3 (0.050): 0.009*"tesla" + 0.008*"nice" + 0.007*"not" + 0.006*"like" + 0.004*"click" + 0.004*"think" + 0.004*"fish" + 0.004*"happy" + 0.003*"look" + 0.003*"elon"
INFO - 21:57:26: topic diff=0.027539, rho=0.100096
INFO - 21:57:26: PROGRESS: pass 1, dispatched chunk #81 = documents up to #164000/195618, outstanding queue size 5
INFO - 21:57:26: PROGRESS: pass 1, dispatched chunk #82 = documents up to #166000/195618, outstanding queue size 4
INFO - 21:57:26: PROGRESS: pass 1, dispatched chunk #83 = documents up to #168000/195618, outstanding queue size 5
INFO - 21:57:26: PROGRESS: pass 1, dispatched c

...

INFO - 21:57:55: PROGRESS: pass 3, dispatched chunk #52 = documents up to #106000/195618, outstanding queue size 5
INFO - 21:57:55: PROGRESS: pass 3, dispatched chunk #53 = documents up to #108000/195618, outstanding queue size 5
INFO - 21:57:56: PROGRESS: pass 3, dispatched chunk #54 = documents up to #110000/195618, outstanding queue size 5
INFO - 21:57:56: PROGRESS: pass 3, dispatched chunk #55 = documents up to #112000/195618, outstanding queue size 4
INFO - 21:57:56: PROGRESS: pass 3, dispatched chunk #56 = documents up to #114000/195618, outstanding queue size 5
INFO - 21:57:56: PROGRESS: pass 3, dispatched chunk #57 = documents up to #116000/195618, outstanding queue size 4
INFO - 21:57:56: PROGRESS: pass 3, dispatched chunk #58 = documents up to #118000/195618, outstanding queue size 4
INFO - 21:57:56: PROGRESS: pass 3, dispatched chunk #59 = documents up to #120000/195618, outstanding queue size 4
INFO - 21:57:57: PROGRESS: pass 3, dispatched chunk #60 = documents up to #12200

INFO - 21:58:06: PROGRESS: pass 3, dispatched chunk #96 = documents up to #194000/195618, outstanding queue size 4
INFO - 21:58:06: PROGRESS: pass 3, dispatched chunk #97 = documents up to #195618/195618, outstanding queue size 4
INFO - 21:58:07: merging changes from 9618 documents into a model of 195618 documents
INFO - 21:58:07: topic #6 (0.050): 0.010*"not" + 0.006*"work" + 0.006*"like" + 0.005*"people" + 0.005*"want" + 0.004*"man" + 0.004*"go" + 0.004*"think" + 0.004*"new" + 0.004*"yesterday"
INFO - 21:58:07: topic #0 (0.050): 0.014*"hold" + 0.012*"silver" + 0.008*"not" + 0.007*"exactly" + 0.005*"tip" + 0.005*"sorry" + 0.004*"tank" + 0.004*"like" + 0.004*"plastic" + 0.004*"sleep"
INFO - 21:58:07: topic #10 (0.050): 0.013*"comment" + 0.011*"retard" + 0.011*"not" + 0.008*"story" + 0.007*"poem" + 0.007*"lmao" + 0.007*"question" + 0.007*"ban" + 0.006*"welcome" + 0.006*"bot"
INFO - 21:58:08: topic #5 (0.050): 0.029*"buy" + 0.018*"spy" + 0.016*"call" + 0.016*"put" + 0.013*"sell" + 0.011*

INFO - 21:59:05: PROGRESS: pass 4, dispatched chunk #50 = documents up to #102000/195618, outstanding queue size 31
INFO - 21:59:05: PROGRESS: pass 4, dispatched chunk #51 = documents up to #104000/195618, outstanding queue size 32
INFO - 21:59:05: PROGRESS: pass 4, dispatched chunk #52 = documents up to #106000/195618, outstanding queue size 33
INFO - 21:59:05: PROGRESS: pass 4, dispatched chunk #53 = documents up to #108000/195618, outstanding queue size 33
INFO - 21:59:06: PROGRESS: pass 4, dispatched chunk #54 = documents up to #110000/195618, outstanding queue size 33
INFO - 21:59:06: PROGRESS: pass 4, dispatched chunk #55 = documents up to #112000/195618, outstanding queue size 33
INFO - 21:59:06: PROGRESS: pass 4, dispatched chunk #56 = documents up to #114000/195618, outstanding queue size 34
INFO - 21:59:06: PROGRESS: pass 4, dispatched chunk #57 = documents up to #116000/195618, outstanding queue size 35
INFO - 21:59:06: PROGRESS: pass 4, dispatched chunk #58 = documents up t

INFO - 21:59:13: PROGRESS: pass 4, dispatched chunk #94 = documents up to #190000/195618, outstanding queue size 26
INFO - 21:59:13: PROGRESS: pass 4, dispatched chunk #95 = documents up to #192000/195618, outstanding queue size 26
INFO - 21:59:13: PROGRESS: pass 4, dispatched chunk #96 = documents up to #194000/195618, outstanding queue size 26
INFO - 21:59:13: PROGRESS: pass 4, dispatched chunk #97 = documents up to #195618/195618, outstanding queue size 26
INFO - 21:59:14: merging changes from 30000 documents into a model of 195618 documents
INFO - 21:59:14: topic #19 (0.050): 0.012*"split" + 0.012*"good" + 0.011*"bro" + 0.008*"not" + 0.007*"cool" + 0.006*"say" + 0.006*"nz" + 0.005*"awesome" + 0.005*"day" + 0.005*"strike"
INFO - 21:59:14: topic #2 (0.050): 0.012*"thanks" + 0.010*"news" + 0.009*"not" + 0.007*"go" + 0.006*"ath" + 0.005*"stonk" + 0.005*"lie" + 0.005*"bird" + 0.005*"like" + 0.004*"read"
INFO - 21:59:14: topic #15 (0.050): 0.025*"yes" + 0.009*"not" + 0.007*"earning" + 0.

[(0,
  '0.011*"hold" + 0.010*"silver" + 0.008*"exactly" + 0.008*"not" + 0.007*"tip" + 0.006*"sorry" + 0.005*"plastic" + 0.005*"sleep" + 0.004*"pay" + 0.004*"beautiful"'),
 (1,
  '0.009*"not" + 0.008*"tomorrow" + 0.007*"think" + 0.005*"go" + 0.005*"like" + 0.004*"ballot" + 0.004*"people" + 0.004*"wrong" + 0.004*"msft" + 0.003*"illegal"'),
 (2,
  '0.012*"thanks" + 0.010*"news" + 0.009*"ath" + 0.009*"not" + 0.007*"go" + 0.007*"stonk" + 0.005*"bear" + 0.005*"print" + 0.005*"lie" + 0.005*"like"'),
 (3,
  '0.027*"tesla" + 0.017*"nice" + 0.009*"click" + 0.009*"elon" + 0.009*"iv" + 0.007*"like" + 0.007*"check" + 0.006*"look" + 0.006*"not" + 0.006*"joke"'),
 (4,
  '0.013*"mask" + 0.010*"wear" + 0.009*"not" + 0.007*"way" + 0.006*"friday" + 0.006*"jesus" + 0.006*"dumb" + 0.005*"okay" + 0.005*"like" + 0.005*"boy"'),
 (5,
  '0.028*"buy" + 0.017*"spy" + 0.017*"call" + 0.015*"put" + 0.015*"sell" + 0.010*"dip" + 0.009*"today" + 0.009*"aapl" + 0.009*"not" + 0.009*"yesterday"'),
 (6,
  '0.010*"not" + 0.

In [18]:
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

I'm noticing a lot of topics from wallstreetbets - about investment/trading terms (robinhood, "bullish", brokerage, volatility, etc), corporations (Uber, Boeing, Nvidia, Tesla), gambling jokes, and a lot of their raunchy/inappropriate language. I also see some topics about politicans and Hong Kong, which makes sense because r/politics is also highly represented in the data (wallstreetbets has 36,719 comments, and politics has 38,821). Interestingly, I don't see many scientific terms, even though r/science has 59,966 comments in the dataset. Maybe that's because the terms used more frequently there are less obvious, just normal words, and the "scientific terms" are so widely varied and more seldom used that they don't appear in topics... the words aren't as recognizable? Most of the other subreddits have lower numbers of comments overall, so it makes sense they wouldn't be represented.

Just for fun, let's try it one last time with r/science.

In [19]:
clean_text = f.pre_clean(science)
lemmatized = f.lemmatize(clean_text)

In [20]:
cvn = TfidfVectorizer(stop_words=stop_list)
data_cvn = cvn.fit_transform(lemmatized)
data_dtmn = pd.DataFrame(data_cvn.toarray(), columns=cvn.get_feature_names()).T
data_dtmn

,0,1,2,3,4,5,6,7,8,9,...,59956,59957,59958,59959,59960,59961,59962,59963,59964,59965
aa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaaayyyyy,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaammmm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaaand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
饮用水reddit,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ﬁrst,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ｓａｓｑｕａｔｃｈｓ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ﾟヮﾟ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(data_dtmn))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in cvn.vocabulary_.items())
word2id = dict((k, v) for k, v in cvn.vocabulary_.items())

In [22]:
ldan = models.LdaMulticore(corpus=corpusn, num_topics=12, id2word=id2wordn, passes=5)
ldan.print_topics()

INFO - 22:19:51: using symmetric alpha at 0.08333333333333333
INFO - 22:19:51: using symmetric eta at 0.08333333333333333
INFO - 22:19:51: using serial LDA version on this node
INFO - 22:19:51: running online LDA training, 12 topics, 5 passes over the supplied corpus of 59966 documents, updating every 30000 documents, evaluating every ~59966 documents, iterating 50x with a convergence threshold of 0.001000
WARNING - 22:19:51: too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
INFO - 22:19:51: training LDA model using 15 processes
INFO - 22:19:58: PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/59966, outstanding queue size 1
INFO - 22:19:58: PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/59966, outstanding queue size 2
INFO - 22:19:58: PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/59966, outstanding queue size 3
INFO - 22:20:02: PROGRESS: pass 0, dispatched chunk #3 = docu

INFO - 22:20:09: merging changes from 30000 documents into a model of 59966 documents
INFO - 22:20:09: topic #8 (0.083): 0.008*"not" + 0.005*"people" + 0.004*"like" + 0.003*"think" + 0.003*"work" + 0.003*"time" + 0.003*"good" + 0.003*"yes" + 0.003*"say" + 0.003*"mean"
INFO - 22:20:09: topic #3 (0.083): 0.008*"not" + 0.006*"tip" + 0.006*"people" + 0.005*"pay" + 0.003*"work" + 0.003*"like" + 0.003*"think" + 0.003*"energy" + 0.003*"live" + 0.003*"say"
INFO - 22:20:09: topic #9 (0.083): 0.008*"not" + 0.004*"people" + 0.004*"need" + 0.004*"think" + 0.004*"like" + 0.003*"study" + 0.003*"problem" + 0.002*"know" + 0.002*"right" + 0.002*"country"
INFO - 22:20:09: topic #0 (0.083): 0.008*"not" + 0.005*"people" + 0.004*"study" + 0.004*"need" + 0.003*"know" + 0.003*"like" + 0.003*"time" + 0.003*"think" + 0.003*"thing" + 0.002*"use"
INFO - 22:20:09: topic #2 (0.083): 0.010*"not" + 0.006*"people" + 0.005*"think" + 0.004*"like" + 0.003*"know" + 0.003*"thing" + 0.003*"say" + 0.003*"look" + 0.003*"year

INFO - 22:20:16: topic #11 (0.083): 0.008*"not" + 0.008*"like" + 0.007*"thank" + 0.005*"know" + 0.005*"good" + 0.005*"people" + 0.004*"think" + 0.004*"study" + 0.003*"way" + 0.003*"thing"
INFO - 22:20:16: topic #9 (0.083): 0.009*"not" + 0.004*"people" + 0.004*"think" + 0.004*"like" + 0.004*"need" + 0.003*"problem" + 0.003*"study" + 0.003*"know" + 0.002*"right" + 0.002*"trump"
INFO - 22:20:16: topic #7 (0.083): 0.013*"not" + 0.005*"people" + 0.004*"think" + 0.004*"like" + 0.004*"agree" + 0.003*"know" + 0.003*"time" + 0.003*"want" + 0.003*"thanks" + 0.003*"thing"
INFO - 22:20:16: topic #0 (0.083): 0.008*"not" + 0.004*"people" + 0.004*"study" + 0.004*"need" + 0.003*"know" + 0.003*"like" + 0.003*"time" + 0.003*"think" + 0.003*"thing" + 0.002*"fish"
INFO - 22:20:16: topic diff=0.084533, rho=0.174123
INFO - 22:20:16: -10.932 per-word bound, 1953.5 perplexity estimate based on a held-out corpus of 1966 documents with 5228 words
INFO - 22:20:16: PROGRESS: pass 3, dispatched chunk #0 = document

INFO - 22:20:22: PROGRESS: pass 4, dispatched chunk #15 = documents up to #32000/59966, outstanding queue size 5
INFO - 22:20:23: PROGRESS: pass 4, dispatched chunk #16 = documents up to #34000/59966, outstanding queue size 5
INFO - 22:20:23: PROGRESS: pass 4, dispatched chunk #17 = documents up to #36000/59966, outstanding queue size 5
INFO - 22:20:23: PROGRESS: pass 4, dispatched chunk #18 = documents up to #38000/59966, outstanding queue size 5
INFO - 22:20:23: merging changes from 30000 documents into a model of 59966 documents
INFO - 22:20:23: topic #9 (0.083): 0.008*"not" + 0.004*"people" + 0.004*"think" + 0.004*"like" + 0.003*"need" + 0.003*"problem" + 0.003*"study" + 0.003*"trump" + 0.003*"know" + 0.002*"love"
INFO - 22:20:23: topic #0 (0.083): 0.008*"not" + 0.004*"people" + 0.004*"study" + 0.004*"need" + 0.003*"like" + 0.003*"know" + 0.003*"time" + 0.003*"think" + 0.002*"fish" + 0.002*"use"
INFO - 22:20:23: topic #4 (0.083): 0.005*"not" + 0.003*"people" + 0.003*"like" + 0.003*

[(0,
  '0.008*"not" + 0.004*"people" + 0.004*"study" + 0.004*"need" + 0.003*"fish" + 0.003*"like" + 0.003*"know" + 0.003*"time" + 0.003*"think" + 0.002*"eat"'),
 (1,
  '0.008*"not" + 0.005*"people" + 0.004*"like" + 0.004*"mask" + 0.004*"go" + 0.003*"think" + 0.003*"thing" + 0.003*"know" + 0.003*"work" + 0.003*"exactly"'),
 (2,
  '0.011*"not" + 0.007*"people" + 0.005*"think" + 0.005*"like" + 0.003*"thing" + 0.003*"know" + 0.003*"life" + 0.003*"year" + 0.003*"go" + 0.003*"say"'),
 (3,
  '0.008*"not" + 0.007*"tip" + 0.007*"pay" + 0.006*"people" + 0.004*"energy" + 0.004*"work" + 0.004*"wage" + 0.003*"like" + 0.003*"live" + 0.003*"restaurant"'),
 (4,
  '0.005*"not" + 0.003*"people" + 0.003*"like" + 0.003*"time" + 0.002*"nice" + 0.002*"good" + 0.002*"know" + 0.002*"go" + 0.002*"great" + 0.002*"think"'),
 (5,
  '0.008*"not" + 0.006*"vaccine" + 0.006*"people" + 0.004*"year" + 0.004*"virus" + 0.004*"long" + 0.003*"think" + 0.003*"work" + 0.003*"like" + 0.003*"study"'),
 (6,
  '0.006*"not" + 0.0

In [23]:
d = corpora.Dictionary()
d.id2token = id2wordn
d.token2id = word2id

vis_data = gensimvis.prepare(ldan, corpusn, d)
pyLDAvis.display(vis_data)

Now I undestand why I didn't recognize any "scientific" looking words in the last topic visualization... they blend in with other topics. The only clearly scientific topic I see here is #6, which includes hydroxychloroquine (to be honest, when I saw hydroxychloroquine in the topic map above this one (the one covering all subreddits besides askreddit) I figured it was from r/politics, but it makes sense it's discussed on the science board. Other words in this topic include staphylococcus, archaeology, tinnitus, rem